In [2]:
import pandas as pd
import numpy as np
import os 

def pobierz_nazwy_plikow(folder):
    pliki = os.listdir(folder)
    pliki_tsv = [plik for plik in pliki]
    return pliki_tsv

def wczytaj_dane(folder, nazwa_pliku, pliki):
    pelna_nazwa = folder + '/' + nazwa_pliku
    tmp = pd.read_csv(pelna_nazwa, sep='\t')
    nazwa_bez_tsv = nazwa_pliku[:-4]  # Usunięcie '.tsv'
    A, B = nazwa_bez_tsv.split('.')
    nazwa = f"{A}_{B}"
    pliki[nazwa] = tmp

folder = 'data'
nazwy_plikow = pobierz_nazwy_plikow(folder)
print(nazwy_plikow)
pliki = {}
for nazwa_pliku in nazwy_plikow:
    wczytaj_dane(folder, nazwa_pliku, pliki)

['title.ratings.tsv', 'title.principals.tsv', 'title.akas.tsv', 'name.basics.tsv', 'title.basics.tsv', 'title.episode.tsv', 'title.crew.tsv']


/var/folders/0r/hq8l80pd28v6wlvr518j1g700000gn/T/ipykernel_26330/3371865516.py:12: DtypeWarning:

Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
klucze = [s[:-4].replace('.', '_') for s in nazwy_plikow]
klucze

['title_ratings',
 'title_principals',
 'title_akas',
 'name_basics',
 'title_basics',
 'title_episode',
 'title_crew']

In [4]:
for i in range(len(klucze)):
    print(pliki[klucze[i]].columns)

Index(['tconst', 'averageRating', 'numVotes'], dtype='object')
Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], dtype='object')
Index(['titleId', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'isOriginalTitle'],
      dtype='object')
Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')
Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')
Index(['tconst', 'parentTconst', 'seasonNumber', 'episodeNumber'], dtype='object')
Index(['tconst', 'directors', 'writers'], dtype='object')


## Task 1

In [5]:
reg = pliki[klucze[2]]
#tabela z regionem
print(reg.head())

     titleId  ordering                      title region language  \
0  tt0000001         1                 Carmencita     \N       \N   
1  tt0000001         2                 Carmencita     DE       \N   
2  tt0000001         3                 Carmencita     US       \N   
3  tt0000001         4  Carmencita - spanyol tánc     HU       \N   
4  tt0000001         5                 Καρμενσίτα     GR       \N   

         types     attributes  isOriginalTitle  
0     original             \N                1  
1           \N  literal title                0  
2  imdbDisplay             \N                0  
3  imdbDisplay             \N                0  
4  imdbDisplay             \N                0  


In [6]:
ocen = pliki[klucze[0]]
#tabela z ocenami
print(ocen.head())

      tconst  averageRating  numVotes
0  tt0000001            5.7      2063
1  tt0000002            5.6       279
2  tt0000003            6.5      2030
3  tt0000004            5.4       180
4  tt0000005            6.2      2796


In [7]:
reg = reg[['titleId', 'title', 'region']]
#usuwanie filmow bez podanego kraju produkcji
reg = reg[reg['region'] != '\\N']

In [8]:

ocen_reg_merged = ocen.merge(reg, left_on='tconst', right_on='titleId', how='left')

# Usuwanie nadmiarowej kolumny 'tconst'
ocen_reg = ocen_reg_merged.drop(columns=['tconst'])

# Wyświetlanie nowej tabeli po operacji merge
print("\nTabela ocen_reg_merged po merge:")
print(ocen_reg_merged.head())



Tabela ocen_reg_merged po merge:
      tconst  averageRating  numVotes    titleId                      title  \
0  tt0000001            5.7      2063  tt0000001                 Carmencita   
1  tt0000001            5.7      2063  tt0000001                 Carmencita   
2  tt0000001            5.7      2063  tt0000001  Carmencita - spanyol tánc   
3  tt0000001            5.7      2063  tt0000001                 Καρμενσίτα   
4  tt0000001            5.7      2063  tt0000001                 Карменсита   

  region  
0     DE  
1     US  
2     HU  
3     GR  
4     RU  


In [9]:
tmp = ocen_reg.copy()
print(tmp.head())

   averageRating  numVotes    titleId                      title region
0            5.7      2063  tt0000001                 Carmencita     DE
1            5.7      2063  tt0000001                 Carmencita     US
2            5.7      2063  tt0000001  Carmencita - spanyol tánc     HU
3            5.7      2063  tt0000001                 Καρμενσίτα     GR
4            5.7      2063  tt0000001                 Карменсита     RU


In [10]:
ocen_reg = tmp.copy()
ocen_reg = ocen_reg[ocen_reg['region'] != '\\N']
s = ocen_reg.sort_values(by=['averageRating'])
print(s.head(20))
#percentile_90 = ocen_reg.groupby('region')['numVotes'].quantile(0.99).reset_index()
percentile_90 = ocen_reg['numVotes'].quantile(0.60)
#percentile_90.columns = ['region', 'treshold']
percentile_90

         averageRating  numVotes     titleId  \
4274719            1.0       112   tt7360990   
2543561            1.0         7  tt14534576   
2543560            1.0         7  tt14534576   
2543559            1.0         7  tt14534576   
2543558            1.0         7  tt14534576   
2543557            1.0         7  tt14534570   
2543556            1.0         7  tt14534570   
2543555            1.0         7  tt14534570   
2543554            1.0         7  tt14534570   
2543553            1.0         7  tt14534570   
2543552            1.0         7  tt14534570   
2543551            1.0         7  tt14534570   
2543550            1.0         7  tt14534568   
2543549            1.0         7  tt14534568   
2543548            1.0         7  tt14534568   
2543547            1.0         7  tt14534568   
2543546            1.0         7  tt14534568   
2543545            1.0         7  tt14534568   
2543544            1.0         7  tt14534568   
2543543            1.0         7  tt1453

341.0

In [13]:
median_votes_per_region = ocen_reg.groupby('region')['numVotes'].median().reset_index()
median_votes_per_region.columns = ['region', 'treshold']
print(median_votes_per_region)
# Merge the median values back to the original DataFrame
ocen_reg = ocen_reg.merge(median_votes_per_region, on='region')


    region  treshold
0       AD      19.0
1       AE    3958.0
2       AF      21.5
3       AG      24.0
4       AI      20.0
..     ...       ...
236   YUCS      75.5
237     ZA    2158.0
238     ZM      11.0
239   ZRCD      17.5
240     ZW      19.0

[241 rows x 2 columns]


In [14]:
#Filter rows where sumVotes is greater than the median for their respective region
filtered_ocen_reg = ocen_reg[ocen_reg['numVotes'] > ocen_reg['treshold']]
filtered_ocen_reg = ocen_reg[ocen_reg['numVotes'] > percentile_90]

# Drop the 'medianSumVotes' column as it's no longer needed
filtered_ocen_reg = filtered_ocen_reg.drop(columns=['treshold'])

print(filtered_ocen_reg.sort_values(by=['numVotes']))

        averageRating  numVotes    titleId                          title  \
912524            5.4       342  tt0123890               Hodja from Pjort   
91876             4.9       342  tt0029006      De zangvogel van Broadway   
91875             4.9       342  tt0029006               La femme en cage   
91874             4.9       342  tt0029006               La femme en cage   
91873             4.9       342  tt0029006             Hitting a New High   
...               ...       ...        ...                            ...   
822189            9.3   2912371  tt0111161                A remény rabjai   
822188            9.3   2912371  tt0111161         À l'ombre de Shawshank   
822187            9.3   2912371  tt0111161              Побег из Шоушенка   
822185            9.3   2912371  tt0111161              Побег из Шоушенка   
822229            9.3   2912371  tt0111161  Vykúpenie z väznice Shawshank   

       region  
912524     US  
91876      BE  
91875      FR  
91874      

In [15]:

# Step 2: Sort the DataFrame by averageRating in descending order
filtered_ocen_reg_sorted = filtered_ocen_reg.sort_values(by='averageRating', ascending=False)
#print(filtered_ocen_reg_sorted)
# Step 3: Initialize the counts dictionary
counts = {region: [] for region in filtered_ocen_reg_sorted['region'].unique()}
top_n_values = range(10, 201, 10)

# # Step 4: Count representation for each region in top N movies
for top_n in top_n_values:
    top_n_filtered_ocen_reg = filtered_ocen_reg_sorted.head(top_n)
    #print(top_n_filtered_ocen_reg)
    region_counts = top_n_filtered_ocen_reg['region'].value_counts()
    print(region_counts)
    for region in counts.keys():
        counts[region].append(region_counts.get(region, 0))

# Step 5: Create the final DataFrame
final_filtered_ocen_reg = pd.DataFrame(counts, index=top_n_values).transpose()
final_filtered_ocen_reg.columns = top_n_values

#print(final_filtered_ocen_reg)

region
TR    6
US    3
JP    1
Name: count, dtype: int64
region
TR    9
US    6
JP    2
IN    1
FR    1
PT    1
Name: count, dtype: int64
region
US    10
TR     9
JP     3
IN     2
IT     2
FR     1
PT     1
GR     1
ES     1
Name: count, dtype: int64
region
US    12
TR     9
JP     5
IN     2
FR     2
IT     2
ES     2
PT     1
GR     1
MX     1
DE     1
GB     1
CA     1
Name: count, dtype: int64
region
US    16
TR    10
JP     9
IN     2
FR     2
IT     2
ES     2
DE     2
PT     1
GR     1
MX     1
GB     1
CA     1
Name: count, dtype: int64
region
US    19
TR    10
JP    10
ES     4
IN     3
FR     3
IT     3
DE     3
PT     1
GR     1
MX     1
GB     1
CA     1
Name: count, dtype: int64
region
US    21
TR    13
JP    12
IN     5
FR     4
ES     4
IT     3
DE     3
PT     1
GR     1
MX     1
GB     1
CA     1
Name: count, dtype: int64
region
US    23
TR    16
JP    15
IN     5
FR     4
IT     4
ES     4
DE     3
PT     2
GR     1
MX     1
GB     1
CA     1
Name: count, dtype: int6

In [16]:
print(final_filtered_ocen_reg)

    10   20   30   40   50   60   70   80   90   100  110  120  130  140  150  \
TR    6    9    9    9   10   10   13   16   16   19   19   19   19   19   20   
US    3    6   10   12   16   19   21   23   25   26   31   32   36   39   42   
JP    1    2    3    5    9   10   12   15   16   16   17   19   20   23   23   
IN    0    1    2    2    2    3    5    5    5    6    6    8    9    9   11   
FR    0    1    1    2    2    3    4    4    4    5    5    6    7    7    9   
..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
HN    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
MU    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
KN    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
GI    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
BM    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   

    160  170  180  190  200

In [22]:
import pandas as pd
import plotly.graph_objects as go

# Normalize counts by table numbers
normalized_df = final_filtered_ocen_reg.div(final_filtered_ocen_reg.columns, axis=1)

# Transpose the DataFrame for easier plotting
normalized_df = normalized_df.transpose()
# Create the plotly figure
fig = go.Figure()

# Add traces for each region
for region in normalized_df.columns:
    fig.add_trace(go.Scatter(
        x=normalized_df.index,
        y=normalized_df[region],
        mode='lines+markers',
        name=region,
        hoverinfo='text',
        text=[f'Kraj: {region}<br>Nr top. filmów: {x}<br>Udział procentowy: {y:.2f}'
              for x, y in zip(normalized_df.index, normalized_df[region])]
    ))

# Update layout
fig.update_layout(
    title='Hegemoni wśród krajów powyzej danej liczby głosów',
    xaxis_title='Udział krajów w topowych filmach',
    yaxis_title='Udział kraju w danej liczbie topowych filmów',
    showlegend=False
)

# Show the plot
fig.show()
print("task2")

## Task 2

### a)

In [290]:

weak_impact = ocen_reg.groupby('region')['numVotes'].sum().reset_index()
weak_impact = weak_impact.sort_values(by=['numVotes'], ascending=False)
weak_impact['id_wi'] = range(len(weak_impact))
weak_impact['id_wi'] += 1


In [291]:
reg = reg[reg['region'] != '\\N']
reg = reg[reg['region'] != 'NaN']
gdp = reg['region'].unique()
print(gdp)
gdp = pd.DataFrame(gdp, columns=['region'])
gdp

['DE' 'PT' 'JP' 'SUHH' 'IT' 'FR' 'ES' 'IN' 'US' 'VE' 'CA' 'BZ' 'XWW' 'PH'
 'SI' 'FI' 'GB' 'XYU' 'NG' 'BR' 'BE' 'ID' 'XWG' 'RO' 'SE' 'LT' 'MX' 'BG'
 'GR' 'CSHH' 'KZ' 'RU' 'AU' 'PL' 'UA' 'DK' 'TW' 'NO' 'EC' 'CH' 'NZ' 'RS'
 'AT' 'CZ' 'KR' 'IE' 'NL' 'ZA' 'SG' 'UY' 'AR' 'CU' 'EG' 'HU' 'DDDE' 'TH'
 'CL' 'CM' 'EE' 'CN' 'TR' 'LK' 'HK' 'XEU' 'AE' 'IR' 'GN' 'CO' 'IS' 'IL'
 'UZ' 'SK' 'PR' 'VN' 'GE' 'TG' 'MC' 'HR' 'MY' 'SA' 'XAS' 'MK' 'LB' 'MN'
 'JM' 'PK' 'RW' 'KE' 'LV' 'MM' 'PE' 'BD' 'TN' 'KW' 'AD' 'YUCS' 'IQ' 'PA'
 'MO' 'GP' 'AL' 'SD' 'GW' 'LI' 'BY' 'AZ' 'CG' 'LU' 'DO' 'ET' 'XKO' 'PY'
 'BF' 'HT' 'PS' 'LR' 'HN' 'CY' 'GH' 'MT' 'BO' 'DZ' 'MA' 'NP' 'AM' 'BA'
 'KG' 'MZ' 'MD' 'FO' 'CR' 'AF' 'BB' nan 'GM' 'ZW' 'SY' 'TT' 'SL' 'ZM' 'KP'
 'SV' 'GT' 'TM' 'CD' 'ME' 'BN' 'VI' 'TJ' 'FJ' 'BJ' 'BH' 'QA' 'IM' 'LA'
 'MW' 'GQ' 'XKV' 'NE' 'AQ' 'SM' 'TZ' 'KH' 'SN' 'PF' 'UG' 'JO' 'BS' 'CSXX'
 'MG' 'VG' 'XPI' 'MU' 'PG' 'SC' 'GA' 'YE' 'XSA' 'CI' 'RE' 'ML' 'AI' 'GI'
 'BUMM' 'XSI' 'AO' 'VA' 'MV' 'MQ' 'VDVN' 'CF' 'LY' 'NI

,region
0,DE
1,PT
2,JP
3,SUHH
4,IT
...,...
243,MS
244,JE
245,NR
246,CC


In [292]:
gdp['region'] = np.random.permutation(gdp['region'])
gdp['id_gdp'] = range(len(gdp))
gdp['id_gdp'] += 1
gdp

,region,id_gdp
0,KR,1
1,KI,2
2,BR,3
3,MQ,4
4,BI,5
...,...,...
243,LC,244
244,EH,245
245,GT,246
246,XSI,247


In [295]:
weak_v_gdp = weak_impact.merge(gdp, left_on='region', right_on='region', how='left')
weak_v_gdp['strong_impact_a'] = weak_v_gdp['id_gdp'] - weak_v_gdp['id_wi']
weak_v_gdp.sort_values(by=['strong_impact_a'])

,region,numVotes,id_wi,id_gdp,strong_impact_a
239,KI,8,240,2,-238
238,CW,8,239,29,-210
212,BI,77,213,5,-208
191,MQ,298,192,4,-188
194,AQ,207,195,20,-175
...,...,...,...,...,...
21,RS,1125834052,22,191,169
47,DK,622846684,48,219,171
65,CSHH,132610812,66,240,174
35,VN,928856810,36,218,182


### b)

In [1]:
import pandas as pd
import plotly.graph_objects as go

# Define the data
data = {
    10:  [6, 3, 1, 0],
    20:  [9, 6, 2, 1],
    30:  [9, 10, 3, 2],
    40:  [9, 12, 5, 2],
    50:  [10, 16, 9, 2],
    60:  [10, 19, 10, 3],
    70:  [13, 21, 12, 5],
    80:  [16, 23, 15, 5],
    90:  [16, 25, 16, 5],
    100: [19, 26, 16, 6],
    110: [19, 31, 17, 6],
    120: [19, 32, 19, 8],
    130: [19, 36, 20, 9],
    140: [19, 39, 23, 9],
    150: [20, 42, 23, 11]
}

# Create the DataFrame
df = pd.DataFrame(data, index=['TR', 'US', 'JP', 'IN'])

# Normalize counts by table numbers
normalized_df = df.div(df.columns, axis=1)

# Transpose the DataFrame for easier plotting
normalized_df = normalized_df.transpose()

# Create the plotly figure
fig = go.Figure()

# Add traces for each region
for region in normalized_df.columns:
    fig.add_trace(go.Scatter(
        x=normalized_df.index,
        y=normalized_df[region],
        mode='lines+markers',
        name=region,
        hoverinfo='text',
        text=[f'Region: {region}<br>Table Number: {x}<br>Normalized Count: {y:.2f}'
              for x, y in zip(normalized_df.index, normalized_df[region])]
    ))

# Update layout
fig.update_layout(
    title='Normalized Count of Regions Over Tables',
    xaxis_title='Table Number',
    yaxis_title='Normalized Count (count / table number)',
    showlegend=False
)

# Show the plot
fig.show()


In [339]:
pip install nbformat


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 78 kB 975 kB/s eta 0:00:01
     |████████████████████████████████| 88 kB 118 kB/s eta 0:00:01
     |████████████████████████████████| 310 kB 46 kB/s eta 0:00:012
     |████████████████████████████████| 60 kB 22 kB/s eta 0:00:016
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
